## Data Preprocessing

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
def completeRTN(row):
    tr = str(int(row["TRACKING_REGION_NUMBER"]))
    tn = str(int(row["TRACKING_NUMBER"]))
    if len(tn) < 7:
        tn = ("0" * (7 - len(tn))) + tn
    return tr + "-" + tn

def preprocess(df, coldef):
    columndef = pd.read_excel(coldef)
    
    # process extraction
    for col in df.columns:
        proc = columndef[columndef["feature"] == col]["proc"].values[0]
        if proc == "A1":
            df["A1"] = (df["A1AM"] == "Y").astype(int)
        elif proc == "B6":
            df["B6"] = (df["B6OTHER"] == "Y").astype(int)
        elif proc == "D1TIME":
            df["D1TIME"] = (df["D1AM"] == "Y").astype(int)
        elif proc == "D2TIME":
            df["D2TIME"] = (df["D2AM"] == "Y").astype(int)
        elif proc == "D3TIME":
            df["D3TIME"] = (df["D3AM"] == "Y").astype(int)
        elif proc == "F12":
            df["F12"] = (df["F12OTHER"] == "Y").astype(int)
    
    for col in df.columns:
        proc = columndef[columndef["feature"] == col]["proc"].values[0]
        
        # deal with Y/N
        if proc == "translate10":
            df[col].replace(to_replace={"Y": 1, "N": 0}, inplace=True)
            df[col] = df[col].astype(int)
        # drop column
        elif proc == "drop":
            df.drop(col, axis=1, inplace=True)
        # to be discussed
        elif proc == "?":
            df.drop(col, axis=1, inplace=True)
        # mostly float, and some str
        elif proc == "floatandstr":
            df.drop(col, axis=1, inplace=True)
        # change the type to float
        elif proc == "float":
            df[col] = df[col].astype(float)
        else:
            df.drop(col, axis=1, inplace=True)

def prepmissing(df):
    attributes = df.columns
    nominalvalues = {}

    df = df.replace('N/A', np.NaN)
    # df = df.replace('?', np.NaN)
    for col in df.columns:
        # deal with missing values
        if sum(pd.isnull(df[col])) != 0 or sum(df[col].isin(["?"])) > 0:
            print("%r column (type: %r): %r null" %(col, df[col].dtype, sum(pd.isnull(df[col]))))
#             if df[col].dtype == "object":
#                 md = df[df[col] != np.NaN][col].mode()[0]
#                 df[col] = df[col].replace(np.NaN, md)
#             else:
#                 mn = df[col].astype(float).mean()
#                 df[col] = df[col].replace(np.NaN, mn)

In [3]:
df_101191 = pd.read_excel("data/191_BWSC101 Release Log Form.xlsx")
df_101592 = pd.read_excel("data/592_BWSC101 Release Log Form.xlsx")
df_101607 = pd.read_excel("data/607_BWSC101 Release Log Form.xlsx")

In [4]:
df_107124 = pd.read_excel("data/124_BWSC107 Tier Classification Transmittal Form.xlsx")
df_107621 = pd.read_excel("data/621_BWSC107 Tier Classification Transmittal Form.xlsx")

In [5]:
# df_101191.dtypes.to_csv("101191.csv")
# df_101592.dtypes.to_csv("101592.csv")
# df_101607.dtypes.to_csv("101607.csv")

In [ ]:
df_107124.ix[:, ['B1', 'B1A', 'B1B', '']]

In [6]:
df_101191["RTN"] = df_101191.apply(completeRTN, axis=1)
df_101592["RTN"] = df_101592.apply(completeRTN, axis=1)
df_101607["RTN"] = df_101607.apply(completeRTN, axis=1)
df_107124["RTN"] = df_107124.apply(completeRTN, axis=1)
df_107621["RTN"] = df_107621.apply(completeRTN, axis=1)

In [7]:
print(df_107124.shape)
print(df_107621.shape)

(3558, 96)
(1880, 129)


In [8]:
df_101191 = df_101191.set_index("RTN")
df_101592 = df_101592.set_index("RTN")
df_101607 = df_101607.set_index("RTN")
df_107124 = df_107124.set_index("RTN")
df_107621 = df_107621.set_index("RTN")

In [9]:
# df_101607 = pd.read_excel("data/607_BWSC101 Release Log Form.xlsx")

In [10]:
df_101607.shape

(7125, 250)

In [11]:
# df_107621 = pd.read_excel("data/621_BWSC107 Tier Classification Transmittal Form.xlsx")

In [18]:
df_107621.shape

(1880, 128)

In [26]:
# select those B1 checked, which means the form is used to identified tier classification
count107 = df_107621['B1A'].groupby(df_107621.index).count()

In [34]:
df_107621.ix['1-0000047', ]

,FORM_DATA_ID,DEP_TRACKING_ID,TR_SUBMIT_DATE,TR_RECEIVED_DATE,TR_SIGN_DATE,TR_CREATE_DATE,TR_PROCESS_DATE,TR_PROCESS_STATUS,A5A_GISLAT,A5B_GISLNG,...,PRP_PERSON_LAST_1,PRP_POSITION,PRP_POSITION_1,PRP_POSITION_2,REG_OBJ_NAME,REG_OBJ_STREET_ADDR,TOWN_NAME,TRACKING_NUMBER,TRACKING_REGION_NUMBER,ZIP_CODE
RTN,,,,,,,,,,,,,,,,,,,,,
1-0000047,4586774,674203,2014-08-04 16:34:37,2014-08-04 17:02:15,2014-08-04 16:28:36,2014-08-04 16:08:21,2014-08-04 17:07:08,EPICS_EXP,42.17712,72.61135,...,PALKOVIC,NaN,NaN,NaN,CALLAWAY FACILITY FMR SPAULDING QUESTOR,425 MEADOW ST,CHICOPEE,47,1,NaN
1-0000047,5906335,854707,2016-08-08 10:58:31,2016-08-08 11:22:17,2016-08-08 10:51:08,2016-08-05 15:24:59,2016-08-08 11:35:09,EPICS_EXP,42.17712,72.61135,...,PALKOVIC,NaN,NaN,NaN,CALLAWAY FACILITY FMR SPAULDING QUESTOR,425 MEADOW ST,CHICOPEE,47,1,NaN


In [29]:
df_107621target = {}
for index in count107:
    if count107[index] > 1:
        
    else:
        if df_107621.ix[index, 'B1'] == 'Y'
            df_107621target[index] = df_107621.ix[index, 'B1A']
        else:
            print("No record.")

2

In [141]:
df_tier = df_101607.join(df_107621[["B1A"]], how="inner")

In [142]:
df_tier.shape

(598, 251)

In [37]:
comparison = pd.read_excel("101607proc.xlsx")

In [99]:
preprocess(df_101607, "101607proc.xlsx")

In [114]:
prepmissing(df_tier)

'H13BVOLUME' column (type: dtype('float64')): 598 null
'H13CVOLUME' column (type: dtype('float64')): 598 null


In [115]:
df_tier.drop("H13BVOLUME", axis=1, inplace=True)
df_tier.drop("H13CVOLUME", axis=1, inplace=True)

In [126]:
df_tier.index.unique()

Index(['1-0000047', '1-0000888', '1-0015788', '1-0016126', '1-0017208',
       '1-0017724', '1-0018869', '1-0019309', '1-0019388', '1-0019404',
       ...
       '4-0026406', '4-0026407', '4-0026426', '4-0026438', '4-0026450',
       '4-0026473', '4-0026492', '4-0026510', '4-0026522', '4-0026525'],
      dtype='object', name='RTN', length=553)

In [ ]:
df_tier

## Modeling

In [116]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [117]:
rf = RandomForestClassifier()

In [127]:
X_train, X_test, y_train, y_test = train_test_split(df_tier.iloc[:, :-1], 
                                                    df_tier.iloc[:, -1], 
                                                    test_size=0.33, 
                                                    random_state=36)

In [128]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [129]:
np.mean(y_test == rf.predict(X_test))

0.81313131313131315